## Setting LLM Model

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
print("Open AI - ",os.getenv("LITELLM_URL"),os.getenv("OPENAI_API_MODEL"), os.getenv("OPENAI_API_EMBEDDING"))
print("OLLAMA  - ",os.getenv("OLLAMA_URL"),os.getenv("OLLAMA_MODEL"))
print("Local OLLAMA - ",os.getenv("OLLAMA_LOCAL_URL"),os.getenv("OLLAMA_LOCAL_MODEL"))

Open AI -  https://litellm.lab.sains.com.my/ gpt-4o-mini text-embedding-ada-002
OLLAMA  -  http://172.26.93.35:888/ llama3.3:70b-instruct-q8_0
Local OLLAMA -  http://localhost:11434/ gemma2


In [6]:
from llama_index.core import Settings

### RUN LLM AS OLLAMA 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-ollama

In [7]:
from llama_index.llms.ollama import Ollama
api_base=os.getenv("OLLAMA_LOCAL_URL")
model=os.getenv("OLLAMA_LOCAL_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=120.0)

# use remote ollam
"""
api_base=os.getenv("OLLAMA_URL")
model=os.getenv("OLLAMA_MODEL")
llm = Ollama(model=model, base_url=api_base,request_timeout=180.0)
"""
#test run
response = llm.complete("What is the capital of France?")
print(response)

HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
The capital of France is **Paris**.  🇫🇷 



### RUN LLM AS OPEN AI 

In [ ]:
#install OPEN AI LLM, skip if already installed
!pipenv install llama-index-llms-openai

In [21]:
from llama_index.llms.openai import OpenAI
api_base=os.getenv("LITELLM_URL")
model=os.getenv("OPENAI_API_MODEL")

Settings.llm = OpenAI(
    model=model,
    api_base = api_base,
    temperature=0.3
)

#resp = Settings.llm.complete("What is the capital of France?")
#print(resp)

## Setting Embedding Model

In [8]:
# use open AI embedding
from llama_index.embeddings.openai import OpenAIEmbedding
api_base=os.getenv("LITELLM_URL")
embedding_model=os.getenv("OPENAI_API_EMBEDDING")

Settings.embed_model = OpenAIEmbedding(
    model_name=embedding_model,
    api_base = api_base,
)
#embed_text = Settings.embed_model.get_text_embedding("hello")
#print(f"{len(embed_text)}, {embed_text}")

In [11]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/paul").load_data()

In [12]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from IPython.display import Markdown, display

In [13]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

In [14]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    SimpleKeywordTableIndex,
)
from llama_index.core import SummaryIndex

In [15]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [16]:
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

HTTP Request: POST https://litellm.lab.sains.com.my/embeddings "HTTP/1.1 200 OK"


In [17]:

list_retriever = summary_index.as_retriever()
vector_retriever = vector_index.as_retriever()
keyword_retriever = keyword_index.as_retriever()

In [18]:
from llama_index.core.tools import RetrieverTool

list_tool = RetrieverTool.from_defaults(
    retriever=list_retriever,
    description=(
        "Will retrieve all context from Paul Graham's essay on What I Worked"
        " On. Don't use if the question only requires more specific context."
    ),
)
vector_tool = RetrieverTool.from_defaults(
    retriever=vector_retriever,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On."
    ),
)
keyword_tool = RetrieverTool.from_defaults(
    retriever=keyword_retriever,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On (using entities mentioned in query)"
    ),
)

In [22]:
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)
from llama_index.core.retrievers import RouterRetriever
from llama_index.core.response.notebook_utils import display_source_node

In [23]:
retriever = RouterRetriever(
    selector=PydanticSingleSelector.from_defaults(llm=Settings.llm),
    retriever_tools=[
        list_tool,
        vector_tool,
    ],
)

In [24]:

# will retrieve all context from the author's life
nodes = retriever.retrieve(
    "Can you give me all the context regarding the author's life?"
)
for node in nodes:
    display_source_node(node)

HTTP Request: POST https://litellm.lab.sains.com.my/chat/completions "HTTP/1.1 200 OK"
Selecting retriever 0: The question asks for all the context regarding the author's life, which aligns with retrieving all context from the essay..


**Node ID:** bd01c614-8a2b-4069-96d2-05993d8ceb24<br>**Similarity:** None<br>**Text:** What I Worked On

February 2021

Before college the two main things I worked on, outside of schoo...<br>

**Node ID:** fd63cd84-c264-4654-b94d-b24e2ec9ee78<br>**Similarity:** None<br>**Text:** It seemed only a matter of time before we'd have Mike, and when I saw Winograd using SHRDLU, it s...<br>

**Node ID:** 3d3ed4fd-3cbe-43bd-9930-b1f7a04d6990<br>**Similarity:** None<br>**Text:** There were some surplus Xerox Dandelions floating around the computer lab at one point. Anyone wh...<br>

**Node ID:** 517e2f5e-bb1b-4e61-a29f-dabbf332f6bf<br>**Similarity:** None<br>**Text:** This was now only weeks away. My nice landlady let me leave my stuff in her attic. I had some mon...<br>

...

**Node ID:** cd7fb8c4-f5ba-4485-bc1f-25589e4d5d52<br>**Similarity:** None<br>**Text:** I was going to do three things: hack, write essays, and work on YC. As YC grew, and I grew more e...<br>

**Node ID:** 91a43a94-fa09-42f4-8db7-4cad4359fb15<br>**Similarity:** None<br>**Text:** The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get...<br>

**Node ID:** b0c12c03-375e-4435-b7a0-1ec65e8bb0ad<br>**Similarity:** None<br>**Text:** That wouldn't have been feasible at the time. McCarthy tested his interpreter by hand-simulating ...<br>

**Node ID:** 1fa9c01f-66c5-483c-bf55-5e3a995981a5<br>**Similarity:** None<br>**Text:** It's the everyday words that differ. So if you string together a lot of abstract concepts with a ...<br>

**Node ID:** e79cc5e1-2db7-41c7-9972-5d8d059f5d65<br>**Similarity:** None<br>**Text:** The YC logo itself is an inside joke: the Viaweb logo had been a white V on a red circle, so I ma...<br>

In [25]:
nodes = retriever.retrieve("What did Paul Graham do after RISD?")
for node in nodes:
    display_source_node(node)

HTTP Request: POST https://litellm.lab.sains.com.my/chat/completions "HTTP/1.1 200 OK"
Selecting retriever 1: The question asks for specific context about Paul Graham's activities after RISD, which aligns with the need for retrieving specific context from his essay..
HTTP Request: POST https://litellm.lab.sains.com.my/embeddings "HTTP/1.1 200 OK"


**Node ID:** 3d3ed4fd-3cbe-43bd-9930-b1f7a04d6990<br>**Similarity:** 0.57754556660963<br>**Text:** There were some surplus Xerox Dandelions floating around the computer lab at one point. Anyone wh...<br>

**Node ID:** ac88b7b2-3937-42c1-bdb9-e6482d8a4ebd<br>**Similarity:** 0.5671975996213687<br>**Text:** Our teacher, professor Ulivi, was a nice guy. He could see I worked hard, and gave me a good grad...<br>

In [27]:
retriever = RouterRetriever(
    selector=PydanticMultiSelector.from_defaults(llm=Settings.llm),
    retriever_tools=[list_tool, vector_tool, keyword_tool],
)

In [28]:
nodes = retriever.retrieve(
    "What were noteable events from the authors time at Interleaf and YC?"
)
for node in nodes:
    display_source_node(node)

HTTP Request: POST https://litellm.lab.sains.com.my/chat/completions "HTTP/1.1 200 OK"
Selecting retriever 1: The question asks for specific context regarding notable events, making this choice appropriate for retrieving that information..
HTTP Request: POST https://litellm.lab.sains.com.my/embeddings "HTTP/1.1 200 OK"
Selecting retriever 2: This choice is also relevant as it allows for retrieving specific context using entities mentioned in the query, which may include Interleaf and YC..
> Starting query: What were noteable events from the authors time at Interleaf and YC?
query keywords: ['interleaf', 'events', 'yc', 'time', 'noteable', 'authors']
> Extracted keywords: ['interleaf', 'yc', 'time']


**Node ID:** e79cc5e1-2db7-41c7-9972-5d8d059f5d65<br>**Similarity:** None<br>**Text:** The YC logo itself is an inside joke: the Viaweb logo had been a white V on a red circle, so I ma...<br>

**Node ID:** ac88b7b2-3937-42c1-bdb9-e6482d8a4ebd<br>**Similarity:** None<br>**Text:** Our teacher, professor Ulivi, was a nice guy. He could see I worked hard, and gave me a good grad...<br>

**Node ID:** 91a43a94-fa09-42f4-8db7-4cad4359fb15<br>**Similarity:** None<br>**Text:** The stroke destroyed her balance, and she was put in a nursing home, but she really wanted to get...<br>

**Node ID:** cd7fb8c4-f5ba-4485-bc1f-25589e4d5d52<br>**Similarity:** None<br>**Text:** I was going to do three things: hack, write essays, and work on YC. As YC grew, and I grew more e...<br>

...

**Node ID:** 22582940-e229-44b2-94f9-d0d522063227<br>**Similarity:** None<br>**Text:** I mistakenly thought the thing that mattered was the absolute number of users. And that is the th...<br>

**Node ID:** 84a92dc0-9c9f-4d8a-9074-1da2740d0083<br>**Similarity:** None<br>**Text:** I stuck it out for a few more months, then in desperation I went back to New York, where unless y...<br>

**Node ID:** b0c12c03-375e-4435-b7a0-1ec65e8bb0ad<br>**Similarity:** None<br>**Text:** That wouldn't have been feasible at the time. McCarthy tested his interpreter by hand-simulating ...<br>

**Node ID:** 517e2f5e-bb1b-4e61-a29f-dabbf332f6bf<br>**Similarity:** None<br>**Text:** This was now only weeks away. My nice landlady let me leave my stuff in her attic. I had some mon...<br>

**Node ID:** fd63cd84-c264-4654-b94d-b24e2ec9ee78<br>**Similarity:** None<br>**Text:** It seemed only a matter of time before we'd have Mike, and when I saw Winograd using SHRDLU, it s...<br>